In [3]:
import pyprind
import pandas as pd
import os

In [4]:
pbar = pyprind.ProgBar(50000)

0%                          100%


In [5]:
labels = {'pos': 1, 'neg': 0}

In [6]:
df = pd.DataFrame()

In [11]:
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = './aclImdb/%s/%s' % (s, l)
        for file in os.listdir(path):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
                df = df.append([[txt, labels[l]]], ignore_index=True)
                pbar.update()

[##############################] | ETA: 00:00:00
Total time elapsed: 00:35:39


In [12]:
df.columns = ['review', 'sentiment']

In [13]:
import numpy as np

In [14]:
np.random.seed(0)

In [15]:
df = df.reindex(np.random.permutation(df.index))

In [16]:
df.to_csv('./movie_data.csv', index=False)

In [18]:
df = pd.read_csv('./movie_data.csv')

In [19]:
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [20]:
from sklearn.feature_extraction.text import CountVectorizer

In [21]:
count = CountVectorizer()

In [22]:
docs = np.array(['The sun is shining', 'The weather is sweet', 'The sun is shining, the weather is sweet, and one and one is two'])

In [23]:
bag = count.fit_transform(docs)

In [24]:
print(count.vocabulary_)

{'shining': 3, 'sun': 4, 'one': 2, 'weather': 8, 'two': 7, 'and': 0, 'the': 6, 'sweet': 5, 'is': 1}


In [25]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


In [26]:
from sklearn.feature_extraction.text import TfidfTransformer

In [28]:
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)

In [29]:
np.set_printoptions(precision=2)

In [30]:
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[ 0.    0.43  0.    0.56  0.56  0.    0.43  0.    0.  ]
 [ 0.    0.43  0.    0.    0.    0.56  0.43  0.    0.56]
 [ 0.5   0.45  0.5   0.19  0.19  0.19  0.3   0.25  0.19]]


In [33]:
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

In [34]:
import re

In [38]:
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    return text

In [36]:
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [39]:
preprocessor("</a>This :) is :( a test :-)!")

'this is a test :) :( :)'

In [40]:
df['review'] = df['review'].apply(preprocessor)

In [41]:
def tokenizer(text):
    return text.split()

In [42]:
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [43]:
from nltk.stem.porter import PorterStemmer

In [44]:
porter = PorterStemmer()

In [45]:
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [46]:
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [47]:
import nltk

In [49]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shuhei.shogen/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [50]:
from nltk.corpus import stopwords

In [52]:
stop = stopwords.words('english')

In [53]:
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:] if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

In [54]:
X_train = df.loc[:25000, 'review'].values

In [55]:
y_train = df.loc[:25000, 'sentiment'].values

In [58]:
X_test = df.loc[25000:, 'review'].values

In [60]:
y_test = df.loc[25000:, 'sentiment'].values

In [63]:
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [65]:
tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)

In [66]:
param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
               {'vect__ngram_range': [(1, 1)],
                'vect__stop_words': [stop, None],
                'vect__tokenizer': [tokenizer, tokenizer_porter],
                'vect__use_idf': [False],
                'vect__norm': [None],
                'clf__penalty': ['l1', 'l2'],
                'clf__C': [1.0, 10.0, 100.0]}
             ]

In [67]:
lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])

In [68]:
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)

In [1]:
# 8.3
import numpy as np
import re
from nltk.corpus import stopwords

In [2]:
stop = stopwords.words('english')

In [3]:
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

In [4]:
def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [6]:
next(stream_docs(path='./movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [7]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [8]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

In [10]:
vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer
                        )

In [11]:
clf = SGDClassifier(loss='log', random_state=1, n_iter=1)

In [12]:
doc_stream = stream_docs(path='./movie_data.csv')

In [13]:
import pyprind

In [14]:
pbar = pyprind.ProgBar(45)

0%                          100%


In [15]:
classes = np.array([0,1])

In [16]:
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

[##############################] | ETA: 00:00:00
Total time elapsed: 00:02:00


In [17]:
X_test, y_test = get_minibatch(doc_stream, size=5000)

In [18]:
X_test = vect.transform(X_test)

In [19]:
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.867


In [20]:
# 9.1
import pickle
import os

In [21]:
dest = os.path.join('movieclassifier', 'pkl_objects')

In [22]:
if not os.path.exists(dest):
    os.makedirs(dest)

In [23]:
pickle.dump(stop,
            open(os.path.join(dest, 'stopwords.pkl'), 'wb'),
            protocol=4
           )

In [24]:
pickle.dump(clf,
            open(os.path.join(dest, 'classifier.pkl'), 'wb'),
            protocol=4
           )

In [25]:
label = {0: 'negative', 1: 'positive'}
example = ['I love this movie']
X = vect.transform(example)

In [26]:
print('Prediction: %s\nProbability: %.2f%%' %\
      (label[clf.predict(X)[0]], np.max(clf.predict_proba(X))*100))

Prediction: positive
Probability: 82.73%


In [28]:
clf2 = pickle.load(open('movieclassifier/pkl_objects/classifier.pkl', 'rb'))

In [29]:
print('Prediction: %s\nProbability: %.2f%%' %\
      (label[clf2.predict(X)[0]], np.max(clf.predict_proba(X))*100))

Prediction: positive
Probability: 82.73%
